In [177]:
#  Geomtric Brownian Motion Program + Black Scholes
#  Written by Graham Besser, John Lewis, & Scott Griffin
#  Quantitative Finance Club  University of Iowa
#  Simulations are from the closed form version of dS = r * S * dt + sigma * S * dZ 
#    Solved with Ito's Lemma

#  Modified Jan 25 2023 - John Lewis
#  Modified Feb 6 2023 to include black scholes and some general cleanup - John Lewis
#  Modified Feb 19 2023 to include multiple option types and dynamic report - Scott Griffin

In [178]:
#Import packages for the computations

import numpy as np  # we will use this package in nearly all programs
import matplotlib.pyplot as plt1
from scipy.stats import norm


In [179]:
#  This is the function to create one simulated stock path
#  Parameters are passed to the function

def priceSim(sharePriceInitial, timeToExpiry, riskFree, divYield, sigma, steps, numberSimulations, optionType='call'):
    """
    #  Compute an array of N simulated price paths (N) using geometric brownian motion
    Inputs
    # sharePriceInitial = Current stock Price
    # strikePrice = Strike Price
    # timeToExpiry = Time to maturity 1 year = 1, 1 month = 1/12
    # riskFree = risk free interest rate
    # divYield = dividend yield
    # sigma = volatility
    # optionType: call or put option
    Output
    # [steps,numberSimulations] Matrix of asset paths
    """
    
    #input checking
    optionTypes = ['call','put']
    if optionType not in optionTypes:
        raise ValueError("Invalid option type. Expected one of: %s" % optionTypes)
    
    # compute the timestep based on input parameters
    dt = timeToExpiry / steps

    # take the natural log of the initial stock price
    lnS = np.log(sharePriceInitial)

    # Since we are not using time varying volatility and risk free rate, compute constant portions of formulas
    nudt = (riskFree - divYield - sigma * sigma * 0.5) * dt
    sigmadt = sigma * np.sqrt(dt)


    #compute the simulated path
    sharePath =  lnS + np.cumsum(( nudt + sigmadt * np.random.normal(size=(steps, numberSimulations))), axis=0)
    
    #matrix of paths
    paths = np.exp(sharePath)
    # insert zero to beginning of each array in matrix
    paths = np.insert(paths, 0, sharePriceInitial, axis=0)
    # make new empty array at end of matrix
    paths = np.insert(paths, steps+1, 0, axis=0)

    # call option or put option
    if optionType == 'call': # call
        for i in range(numberSimulations):
            paths[steps + 1,i] = np.maximum(paths[steps,i]-strikePrice,0)
    else: # put
        for i in range(numberSimulations):
            paths[steps + 1,i] = np.maximum(strikePrice-paths[steps,i],0)
    
    GBMOptionCost = (np.exp(-riskFree * timeToExpiry) * (sum(paths[steps+1,:])/numberSimulations))
    
    return GBMOptionCost


In [180]:
#  Function for Black Scholes pricing of a simple call option
#  

def simpleBlackScholes(sharePriceInitial, strikePrice, timeToExpiry, riskFree, sigma, optionType='call', return_Nd1_and_Nd2:bool=False):
    """
    #  Compute price of a call option without dividends using Black Scholes
    Inputs
    # sharePriceInitial = Current stock Price
    # strikePrice = Strike Price
    # timeToExpiry = Time to maturity 1 year = 1, 1 month = 1/12
    # riskFree = risk free interest rate
    # sigma = volatility
    # optionType: call or put option
    # return_Nd1_and_Nd2:bool = if True, function returns a list of N(d1), N(d2), and the price

    Output
    # Option price under parameters provided above
    """
    #Input checking-----------------------------------------------------------------------
    optionTypes = ['call','put']
    if optionType not in optionTypes:
        raise ValueError("Invalid option type. Expected one of: %s" % optionTypes)    
    #-------------------------------------------------------------------------------------    
        
    #TODO: put in logic for a call or a put
    
    # N(d1) is the factor by which the discounted expected value of contingent receipt of the stock exceeds the current value of the stock.
    #  In simple terms how far into the money the option is if it is in the money
    d1 = (np.log(sharePriceInitial / strikePrice) + (riskFree + sigma * sigma * 0.5) * timeToExpiry) / (sigma * np.sqrt(timeToExpiry))
     
    # N(d2) is the probability that the option expires in the money    
    d2 = d1 - sigma * np.sqrt(timeToExpiry)
                                                                                                        
    if optionType == 'call': # call price calculation
        blackScholesPrice = sharePriceInitial *  norm.cdf(d1) - strikePrice * np.exp(-riskFree * timeToExpiry) * norm.cdf(d2) 
    else: # put price calcualtion
        blackScholesPrice = strikePrice * np.exp(-riskFree * timeToExpiry) * norm.cdf(-1*d2) - sharePriceInitial *  norm.cdf(-1*d1)
        
    if return_Nd1_and_Nd2 == True:
        return [norm.cdf(d1),norm.cdf(d2),blackScholesPrice]
    
    return(blackScholesPrice)                                                                                                    

In [181]:
# Main program
# Setup parameters for stock and option
# Assume a call option

sharePriceInitial = 49.0  # stock price S_{0}
strikePrice = 49.0  # strike
timeToExpiry = 0.25  # time to maturity
riskFree = 0.047  # risk free risk in annual %
divYield = 0.00  # annual dividend rate
sigma = 0.20  # annual volatility in %
steps = 10  # time steps
numberSimulations = 35000  # number of simulations


In [182]:
# 1. Sensitivty of Black-Scholes model, noting changes in N(d1), N(d2), and price

# need value much lower (x0.5), current value, and value much higher (x2)
# parametersAndResults: dictionary of each parameter stores a dictionary of each of its variations and their resulting N(d1), N(d2) and price
parametersAndResults = {
    sigma:{'name':'sigma',sigma/2:[],sigma:[],sigma*2:[]},
    riskFree:{'name':'riskFree',riskFree/2:[],riskFree:[],riskFree*2:[]},
    timeToExpiry:{'name':'timeToExpiry',timeToExpiry/2:[],timeToExpiry:[],timeToExpiry*2:[]},
    strikePrice:{'name':'strikePrice',strikePrice/2:[],strikePrice:[],strikePrice*2:[]},
}

# for each parameter, vary the original value and store results of black-scholes model
for parameter in parametersAndResults:
    
    if parametersAndResults[parameter]['name'] == 'sigma':
        for variation in parametersAndResults[parameter]:
            if variation=='name':
                continue
            sigma = variation
            parametersAndResults[parameter][variation] = simpleBlackScholes(sharePriceInitial, strikePrice, timeToExpiry, riskFree, sigma, optionType='call', return_Nd1_and_Nd2=True)
        sigma = parameter
        
    elif parametersAndResults[parameter]['name'] == 'riskFree':
        for variation in parametersAndResults[parameter]:
            if variation=='name':
                continue
            riskFree = variation
            parametersAndResults[parameter][variation] = simpleBlackScholes(sharePriceInitial, strikePrice, timeToExpiry, riskFree, sigma, optionType='call', return_Nd1_and_Nd2=True)
        riskFree = parameter
        
    elif parametersAndResults[parameter]['name'] == 'timeToExpiry':
        for variation in parametersAndResults[parameter]:
            if variation=='name':
                continue
            timeToExpiry = variation
            parametersAndResults[parameter][variation] = simpleBlackScholes(sharePriceInitial, strikePrice, timeToExpiry, riskFree, sigma, optionType='call', return_Nd1_and_Nd2=True)
        timeToExpiry = parameter

    elif parametersAndResults[parameter]['name'] == 'strikePrice':
        for variation in parametersAndResults[parameter]:
            if variation=='name':
                continue
            strikePrice = variation
            parametersAndResults[parameter][variation] = simpleBlackScholes(sharePriceInitial, strikePrice, timeToExpiry, riskFree, sigma, optionType='call', return_Nd1_and_Nd2=True)
        strikePrice = parameter

# print initial parameters        
print(f'''    
    Inputs
    ---------------------------
    sharePriceInitial = {sharePriceInitial}
    strikePrice = {strikePrice}
    timeToExpiry = {timeToExpiry}
    riskFree = {riskFree}
    divYield = {divYield}
    sigma = {sigma}
    steps = {steps}
    numberSimulations = {numberSimulations}
    ---------------------------
''')

# generate report of results
report = '''1. Sensitivty of Black-Scholes model, noting changes in N(d1), N(d2), and price 
'''
for parameter in parametersAndResults:
    tempReport = f'''
    {parametersAndResults[parameter]['name']}
    -----------'''
    for variation in parametersAndResults[parameter]:
        if variation=='name':
                continue
        tempReport += f'''
        {variation}: N(d1)={round(parametersAndResults[parameter][variation][0],4)}, N(d2)={round(parametersAndResults[parameter][variation][1],4)}, Price={round(parametersAndResults[parameter][variation][2],2)}'''
    report += tempReport
    
print(report)

# 2. Sensitivity of GBM simulation to sigma and number of steps

parametersAndResults = {
    sigma:{'name':'sigma',sigma/2:None,sigma:None,sigma*2:None},
    steps:{'name':'steps',int(steps/2):None,int(steps):None,int(steps*2):None},
}

# for each parameter, vary the original value and store results of GBM simulation
for parameter in parametersAndResults:
    
    if parametersAndResults[parameter]['name'] == 'sigma':
        for variation in parametersAndResults[parameter]:
            if variation=='name':
                continue
            sigma = variation
            parametersAndResults[parameter][variation] = priceSim(sharePriceInitial, timeToExpiry, riskFree, divYield, sigma, steps, numberSimulations)
        sigma = parameter
    
    elif parametersAndResults[parameter]['name'] == 'steps':
        for variation in parametersAndResults[parameter]:
            if variation=='name':
                continue
            steps = variation
            parametersAndResults[parameter][variation] = priceSim(sharePriceInitial, timeToExpiry, riskFree, divYield, sigma, steps, numberSimulations)
        steps = parameter


# generate report of results
report = "\n2. Sensitivity of GBM simulation to sigma and number of steps\n"
for parameter in parametersAndResults:
    tempReport = f'''
    {parametersAndResults[parameter]['name']}
    -----------'''
    for variation in parametersAndResults[parameter]:
        if variation=='name':
                continue
        tempReport += f'''
        {variation}: Price={round(parametersAndResults[parameter][variation],2)}'''
    report += tempReport
    
print(report)

# 3. Calculate call and put price using black-scholes

callPrice = simpleBlackScholes(sharePriceInitial, strikePrice, timeToExpiry, riskFree, sigma, optionType='call')
putPrice = simpleBlackScholes(sharePriceInitial, strikePrice, timeToExpiry, riskFree, sigma, optionType='put')
report = f'''\n3. Calculate call and put price using black-scholes

    Call price = {round(callPrice,2)}
    Put price = {round(putPrice,2)}'''

print(report)

# 4. Calulate call and put price using GBM
callPrice = priceSim(sharePriceInitial, timeToExpiry, riskFree, divYield, sigma, steps, numberSimulations, optionType='call')
putPrice = priceSim(sharePriceInitial, timeToExpiry, riskFree, divYield, sigma, steps, numberSimulations, optionType='put')
report = f'''\n4. Calulate call and put price using GBM

    Call price = {round(callPrice,2)}
    Put price = {round(putPrice,2)}'''

print(report)


    
    Inputs
    ---------------------------
    sharePriceInitial = 49.0
    strikePrice = 49.0
    timeToExpiry = 0.25
    riskFree = 0.047
    divYield = 0.0
    sigma = 0.2
    steps = 10
    numberSimulations = 35000
    ---------------------------

1. Sensitivty of Black-Scholes model, noting changes in N(d1), N(d2), and price 

    sigma
    -----------
        0.1: N(d1)=0.6026, N(d2)=0.5832, Price=1.28
        0.2: N(d1)=0.5665, N(d2)=0.5269, Price=2.24
        0.4: N(d1)=0.5631, N(d2)=0.4835, Price=4.17
    riskFree
    -----------
        0.0235: N(d1)=0.5433, N(d2)=0.5035, Price=2.1
        0.047: N(d1)=0.5665, N(d2)=0.5269, Price=2.24
        0.094: N(d1)=0.6122, N(d2)=0.5734, Price=2.55
    timeToExpiry
    -----------
        0.125: N(d1)=0.5471, N(d2)=0.519, Price=1.53
        0.25: N(d1)=0.5665, N(d2)=0.5269, Price=2.24
        0.5: N(d1)=0.5936, N(d2)=0.538, Price=3.34
    strikePrice
    -----------
        24.5: N(d1)=1.0, N(d2)=1.0, Price=24.79
        49.0: N(d